In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from functools import reduces

In [2]:
# import numpy as np
# import pandas as pd
# import xarray

# glct = xarray.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT.NC", engine="netcdf4")

# glct5mins_trans = np.zeros((26, 2160, 4320), dtype='<U64')

# label_map = {
#     1: 'agri', 2: 'grass', 3: 'carb', 4: 'rfab', 5: 'rftm',
#     6: 'biof', 7: 'ice', 8: 'tund', 9: 'tuwd', 10: 'bore',
#     11: 'ccfo', 12: 'tmfo', 13: 'tdfo', 14: 'wmfo', 15: 'stepp',
#     16: 'dsrt', 17: 'scrb', 18: 'svna', 19: 'trow', 20: 'trof'
# }

# # Mengisi nilai NaN dengan nilai yang dapat diabaikan
# glct_filled = glct['GLCT'].fillna(0)
# glct_labels = glct_filled.values.astype(int)
# glct_class = np.array([label_map.get(label_id, 'Ocean') for label_id in glct_labels.ravel()]).reshape(glct_labels.shape)

# # for i in range(26):
# #     prev_nicknames_with_to = np.char.add(glct_class[i], '_to_')
# #     glct5mins_trans[i] = np.char.add(prev_nicknames_with_to, glct_class[i+1])

# glct_trans_netcdf = xarray.Dataset({
#         "GLCT_trans":(["time", "latitude", "longitude"], glct_class)
#     },
#     coords={
#         "time": pd.date_range(start='1970-01-01', end='2100-01-01', freq='5YS'),
#         "latitude": glct.coords["latitude"].to_numpy(),
#         "longitude": glct.coords["longitude"].to_numpy()
#     })

# glct_trans_netcdf.to_netcdf("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_nick.NC", mode="w", format="NETCDF4")


In [3]:
glct = xr.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_nick.NC", engine="netcdf4")
garea = xr.open_dataset("/vol/milkunC/achaidir/IMAGE PBL/SSP2/GAREA.NC", engine="netcdf4")
df_sankey = pd.read_excel("/vol/milkunC/achaidir/IMAGE PBL/Konsep Sankey Diagram.xlsx", sheet_name="color")
df_additional = pd.read_excel("/vol/milkunC/achaidir/IMAGE PBL/GLCT_sankey.xlsx", sheet_name="sankey2") 

In [4]:
df_sankey['color'] = '#' + df_sankey['color'].astype(str)
df_sankey.head()

,GLCT_trans,color
0,agri,#F3CA52
1,grass,#DC5F00
2,carb,#F6E9B2
3,rfab,#254336
4,rftm,#6B8A7A


In [5]:
garea_2020 = garea.isel(time=slice(0, 11)) #slice 1975 to 2100
garea_ha = np.nan_to_num(np.multiply(garea_2020['GAREA'].to_numpy(), 100))

glct_2020 = glct.isel(time=slice(0, 11))

In [6]:
country_code = pd.read_excel("/vol/milkunC/achaidir/Country Grids/ISO-3166-Country-Code_Final.xlsx")
luh_static = xr.open_dataset("/vol/milkunC/achaidir/Country Grids/gpw-v4-national-identifier-grid-rev11_2pt5_min_tif_NEW/CCODE_RASTER.nc", engine="netcdf4")

In [7]:
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')
ccode_convert = np.zeros((2160, 4320), dtype="<U64") #rubah ke <U64

In [8]:
ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

for i in range(2160):
    for j in range(4320):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"

country = xr.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": glct_2020.coords["longitude"].to_numpy(), "latitude": glct_2020.coords["latitude"].to_numpy()})
glct_2020_country = glct_2020.assign_coords(country=country.country)

In [9]:
glct_2020_country

<xarray.Dataset>
Dimensions:     (time: 11, latitude: 2160, longitude: 4320)
Coordinates:
  * time        (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2020-01-01
  * latitude    (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude   (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
    country     (latitude, longitude) <U64 'ocean' 'ocean' ... 'ocean' 'ocean'
Data variables:
    GLCT_trans  (time, latitude, longitude) object ...

In [10]:
garea_ha_netcdf = xr.Dataset({
        "GAREA_ha":(["time", "latitude", "longitude"], garea_ha)
    },
    coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "latitude": glct.coords["latitude"].to_numpy(),
        "longitude": glct.coords["longitude"].to_numpy(),
    })
garea_ha_netcdf

<xarray.Dataset>
Dimensions:    (time: 11, latitude: 2160, longitude: 4320)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2020-01-01
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
Data variables:
    GAREA_ha   (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [11]:
df = glct_2020_country['GLCT_trans'].isel(time=0).to_dataframe()
df1 = glct_2020_country['GLCT_trans'].isel(time=1).to_dataframe()
df2 = glct_2020_country['GLCT_trans'].isel(time=2).to_dataframe()
df3 = glct_2020_country['GLCT_trans'].isel(time=3).to_dataframe()
df4 = glct_2020_country['GLCT_trans'].isel(time=4).to_dataframe()
df5 = glct_2020_country['GLCT_trans'].isel(time=5).to_dataframe()
df6 = glct_2020_country['GLCT_trans'].isel(time=6).to_dataframe()
df7 = glct_2020_country['GLCT_trans'].isel(time=7).to_dataframe()
df8 = glct_2020_country['GLCT_trans'].isel(time=8).to_dataframe()
df9 = glct_2020_country['GLCT_trans'].isel(time=9).to_dataframe()
df10 = glct_2020_country['GLCT_trans'].isel(time=10).to_dataframe()

In [12]:
df = df[df['GLCT_trans'] != 'ocean']
df1 = df1[df1['GLCT_trans'] != 'ocean']
df2 = df2[df2['GLCT_trans'] != 'ocean']
df3 = df3[df3['GLCT_trans'] != 'ocean']
df4 = df4[df4['GLCT_trans'] != 'ocean']
df5 = df5[df5['GLCT_trans'] != 'ocean']
df6 = df6[df6['GLCT_trans'] != 'ocean']
df7 = df7[df7['GLCT_trans'] != 'ocean']
df8 = df8[df8['GLCT_trans'] != 'ocean']
df9 = df9[df9['GLCT_trans'] != 'ocean']
df10 = df10[df10['GLCT_trans'] != 'ocean']

In [13]:
df_value = garea_ha_netcdf['GAREA_ha'].isel(time=0).to_dataframe()
df_value1 = garea_ha_netcdf['GAREA_ha'].isel(time=1).to_dataframe()
df_value2 = garea_ha_netcdf['GAREA_ha'].isel(time=2).to_dataframe()
df_value3 = garea_ha_netcdf['GAREA_ha'].isel(time=3).to_dataframe()
df_value4 = garea_ha_netcdf['GAREA_ha'].isel(time=4).to_dataframe()
df_value5 = garea_ha_netcdf['GAREA_ha'].isel(time=5).to_dataframe()
df_value6 = garea_ha_netcdf['GAREA_ha'].isel(time=6).to_dataframe()
df_value7 = garea_ha_netcdf['GAREA_ha'].isel(time=7).to_dataframe()
df_value8 = garea_ha_netcdf['GAREA_ha'].isel(time=8).to_dataframe()
df_value9 = garea_ha_netcdf['GAREA_ha'].isel(time=9).to_dataframe()
df_value10 = garea_ha_netcdf['GAREA_ha'].isel(time=10).to_dataframe()

In [14]:
table = df.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table1 = df1.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table2 = df2.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table3 = df3.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table4 = df4.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table5 = df5.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table6 = df6.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table7 = df7.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table8 = df8.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum') 
table9 = df9.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum')
table10 = df10.pivot_table(index=['time', 'longitude', 'latitude', 'country'], values='GLCT_trans', aggfunc='sum')

In [15]:
table10

GLCT_trans
time       longitude   latitude   country           
2020-01-01 -179.958333 -89.958333 ocean        Ocean
                       -89.874999 ocean        Ocean
                       -89.791666 ocean        Ocean
                       -89.708333 ocean        Ocean
                       -89.624999 ocean        Ocean
...                                              ...
            179.958332  89.625000 ocean        Ocean
                        89.708333 ocean        Ocean
                        89.791667 ocean        Ocean
                        89.875000 ocean        Ocean
                        89.958333 ocean        Ocean

[9331200 rows x 1 columns]

In [16]:
table_value = df_value.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value1 = df_value1.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value2 = df_value2.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value3 = df_value3.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value4 = df_value4.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value5 = df_value5.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value6 = df_value6.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value7 = df_value7.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value8 = df_value8.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum') 
table_value9 = df_value9.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum')
table_value10 = df_value10.pivot_table(index=['time', 'longitude', 'latitude'], values='GAREA_ha', aggfunc='sum')

In [17]:
table_index = table.reset_index()
table1_index = table1.reset_index()
table2_index = table2.reset_index()
table3_index = table3.reset_index()
table4_index = table4.reset_index()
table5_index = table5.reset_index()
table6_index = table6.reset_index()
table7_index = table7.reset_index()
table8_index = table8.reset_index()
table9_index = table9.reset_index()
table10_index = table10.reset_index()

In [18]:
table_index

,time,longitude,latitude,country,GLCT_trans
0,1970-01-01,-179.958333,-89.958333,ocean,Ocean
1,1970-01-01,-179.958333,-89.874999,ocean,Ocean
2,1970-01-01,-179.958333,-89.791666,ocean,Ocean
3,1970-01-01,-179.958333,-89.708333,ocean,Ocean
4,1970-01-01,-179.958333,-89.624999,ocean,Ocean
...,...,...,...,...,...
9331195,1970-01-01,179.958332,89.625000,ocean,Ocean
9331196,1970-01-01,179.958332,89.708333,ocean,Ocean
9331197,1970-01-01,179.958332,89.791667,ocean,Ocean
9331198,1970-01-01,179.958332,89.875000,ocean,Ocean


In [19]:
table_value = table_value.reset_index()
table1_value = table_value1.reset_index()
table2_value = table_value2.reset_index()
table3_value = table_value3.reset_index()
table4_value = table_value4.reset_index()
table5_value = table_value5.reset_index()
table6_value = table_value6.reset_index()
table7_value = table_value7.reset_index()
table8_value = table_value8.reset_index()
table9_value = table_value9.reset_index()
table10_value = table_value10.reset_index()

In [20]:
table_merge = pd.merge(left=table_index, right=table_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table1_merge = pd.merge(left=table1_index, right=table1_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table2_merge = pd.merge(left=table2_index, right=table2_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table3_merge = pd.merge(left=table3_index, right=table3_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table4_merge = pd.merge(left=table4_index, right=table4_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table5_merge = pd.merge(left=table5_index, right=table5_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table6_merge = pd.merge(left=table6_index, right=table6_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table7_merge = pd.merge(left=table7_index, right=table7_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table8_merge = pd.merge(left=table8_index, right=table8_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table9_merge = pd.merge(left=table9_index, right=table9_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])
table10_merge = pd.merge(left=table10_index, right=table10_value, left_on=['time', 'longitude', 'latitude'], right_on=['time', 'longitude', 'latitude'])

In [21]:
table10_merge

,time,longitude,latitude,country,GLCT_trans,GAREA_ha
0,2020-01-01,-179.958333,-89.958333,ocean,Ocean,0.0
1,2020-01-01,-179.958333,-89.874999,ocean,Ocean,0.0
2,2020-01-01,-179.958333,-89.791666,ocean,Ocean,0.0
3,2020-01-01,-179.958333,-89.708333,ocean,Ocean,0.0
4,2020-01-01,-179.958333,-89.624999,ocean,Ocean,0.0
...,...,...,...,...,...,...
9331195,2020-01-01,179.958332,89.625000,ocean,Ocean,0.0
9331196,2020-01-01,179.958332,89.708333,ocean,Ocean,0.0
9331197,2020-01-01,179.958332,89.791667,ocean,Ocean,0.0
9331198,2020-01-01,179.958332,89.875000,ocean,Ocean,0.0


In [22]:
table_color = pd.merge(left=table_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table1_color = pd.merge(left=table1_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table2_color = pd.merge(left=table2_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table3_color = pd.merge(left=table3_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table4_color = pd.merge(left=table4_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table5_color = pd.merge(left=table5_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table6_color = pd.merge(left=table6_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table7_color = pd.merge(left=table7_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table8_color = pd.merge(left=table8_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table9_color = pd.merge(left=table9_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])
table10_color = pd.merge(left=table10_merge, right=df_sankey, left_on=['GLCT_trans'], right_on=['GLCT_trans'])

In [23]:
data = pd.to_datetime(table_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table_color['time'] = data.year

data1 = pd.to_datetime(table1_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table1_color['time'] = data1.year

data2 = pd.to_datetime(table2_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table2_color['time'] = data2.year

data3 = pd.to_datetime(table3_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table3_color['time'] = data3.year

data4 = pd.to_datetime(table4_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table4_color['time'] = data4.year

data5 = pd.to_datetime(table5_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table5_color['time'] = data5.year

data6 = pd.to_datetime(table6_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table6_color['time'] = data6.year

data7 = pd.to_datetime(table7_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table7_color['time'] = data7.year

data8 = pd.to_datetime(table8_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table8_color['time'] = data8.year

data9 = pd.to_datetime(table9_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table9_color['time'] = data9.year

data10 = pd.to_datetime(table10_color['time'].values, format='%d/%m/%Y %H.%M.%S')
table10_color['time'] = data10.year

In [24]:
table_color['GLCT_trans'] = table_color['GLCT_trans'] + table_color['time'].astype(str)
table1_color['GLCT_trans'] = table1_color['GLCT_trans'] + table1_color['time'].astype(str)
table2_color['GLCT_trans'] = table2_color['GLCT_trans'] + table2_color['time'].astype(str)
table3_color['GLCT_trans'] = table3_color['GLCT_trans'] + table3_color['time'].astype(str)
table4_color['GLCT_trans'] = table4_color['GLCT_trans'] + table4_color['time'].astype(str)
table5_color['GLCT_trans'] = table5_color['GLCT_trans'] + table5_color['time'].astype(str)
table6_color['GLCT_trans'] = table6_color['GLCT_trans'] + table6_color['time'].astype(str)
table7_color['GLCT_trans'] = table7_color['GLCT_trans'] + table7_color['time'].astype(str)
table8_color['GLCT_trans'] = table8_color['GLCT_trans'] + table8_color['time'].astype(str)
table9_color['GLCT_trans'] = table9_color['GLCT_trans'] + table9_color['time'].astype(str)
table10_color['GLCT_trans'] = table10_color['GLCT_trans'] + table10_color['time'].astype(str)

In [25]:
table_columns = table_color.drop(columns=['time'])
table1_columns = table1_color.drop(columns=['time'])
table2_columns = table2_color.drop(columns=['time'])
table3_columns = table3_color.drop(columns=['time'])
table4_columns = table4_color.drop(columns=['time'])
table5_columns = table5_color.drop(columns=['time'])
table6_columns = table6_color.drop(columns=['time'])
table7_columns = table7_color.drop(columns=['time'])
table8_columns = table8_color.drop(columns=['time'])
table9_columns = table9_color.drop(columns=['time'])
table10_columns = table10_color.drop(columns=['time'])

In [26]:
table_columns = table_columns.rename(columns={'GLCT_trans' : 'GLCT1970', "color" : "color1970", "GAREA_ha": "GAREA1970"})
table1_columns = table1_columns.rename(columns={'GLCT_trans' : 'GLCT1975', "color" : "color1975", "GAREA_ha": "GAREA1975"})
table2_columns = table2_columns.rename(columns={'GLCT_trans' : 'GLCT1980', "color" : "color1980", "GAREA_ha": "GAREA1980"})
table3_columns = table3_columns.rename(columns={'GLCT_trans' : 'GLCT1985', "color" : "color1985", "GAREA_ha": "GAREA1985"})
table4_columns = table4_columns.rename(columns={'GLCT_trans' : 'GLCT1990', "color" : "color1990", "GAREA_ha": "GAREA1990"})
table5_columns = table5_columns.rename(columns={'GLCT_trans' : 'GLCT1995', "color" : "color1995", "GAREA_ha": "GAREA1995"})
table6_columns = table6_columns.rename(columns={'GLCT_trans' : 'GLCT2000', "color" : "color2000", "GAREA_ha": "GAREA2000"})
table7_columns = table7_columns.rename(columns={'GLCT_trans' : 'GLCT2005', "color" : "color2005", "GAREA_ha": "GAREA2005"})
table8_columns = table8_columns.rename(columns={'GLCT_trans' : 'GLCT2010', "color" : "color2010", "GAREA_ha": "GAREA2010"})
table9_columns = table9_columns.rename(columns={'GLCT_trans' : 'GLCT2015', "color" : "color2015", "GAREA_ha": "GAREA2015"})
table10_columns = table10_columns.rename(columns={'GLCT_trans' : 'GLCT2020', "color" : "color2020", "GAREA_ha": "GAREA2020"})

In [27]:
table10_columns

,longitude,latitude,country,GLCT2020,GAREA2020,color2020
0,-179.958333,-16.875000,Fiji,trof2020,0.000000,#799351
1,-179.958333,-16.791666,Fiji,trof2020,0.000000,#799351
2,-179.958333,-16.541666,Fiji,trof2020,2845.649902,#799351
3,-179.875000,-16.791666,Fiji,trof2020,0.000000,#799351
4,-179.875000,-16.708333,Fiji,trof2020,0.000000,#799351
...,...,...,...,...,...,...
2215824,122.541665,45.875000,China,biof2020,5978.029785,#F2613F
2215825,122.624999,43.541667,China,biof2020,6210.703125,#F2613F
2215826,122.624999,45.791667,China,biof2020,5986.979980,#F2613F
2215827,122.624999,45.875000,China,biof2020,5978.029785,#F2613F


In [28]:
frames = [table_columns, table1_columns, table2_columns, table3_columns, table4_columns, table5_columns, table6_columns, table7_columns, table8_columns, table9_columns, table10_columns]

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['longitude','latitude','country'], how='outer'), frames)
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
merged_df = merged_df.fillna(0)
merged_df

,longitude,latitude,country,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,GLCT1980,...,color2005,GLCT2010,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020
0,-179.958333,-16.875000,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
1,-179.958333,-16.791666,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
2,-179.958333,-16.541666,Fiji,trof1970,2845.649902,#799351,trof1975,2845.649902,#799351,trof1980,...,#799351,trof2010,2845.649902,#799351,trof2015,2845.649902,#799351,trof2020,2845.649902,#799351
3,-179.875000,-16.791666,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
4,-179.875000,-16.708333,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215824,152.124999,-24.791666,Australia,svna1970,7794.999512,#D10363,svna1975,7794.999512,#D10363,svna1980,...,#F3CA52,agri2010,7794.999512,#F3CA52,agri2015,7794.999512,#F3CA52,agri2020,7794.999512,#F3CA52
2215825,152.124999,-24.708333,Australia,svna1970,7800.229492,#D10363,svna1975,7800.229492,#D10363,svna1980,...,#D10363,svna2010,7800.229492,#D10363,svna2015,7800.229492,#D10363,svna2020,7800.229492,#D10363
2215826,152.124999,-24.625000,Australia,svna1970,5697.569824,#D10363,svna1975,5697.569824,#D10363,svna1980,...,#D10363,svna2010,5697.569824,#D10363,svna2015,5697.569824,#D10363,svna2020,5697.569824,#D10363
2215827,152.374999,-24.708333,Australia,svna1970,606.515991,#D10363,svna1975,606.515991,#D10363,svna1980,...,#D10363,svna2010,606.515991,#D10363,svna2015,606.515991,#D10363,svna2020,606.515991,#D10363


In [29]:
ccode_iso = pd.read_excel("/vol/milkunC/achaidir/Country Grids/ISO-3166-Country-Code_Final.xlsx", engine="openpyxl")
ccode_iso

,ISO Country,alpha-2,alpha-3,country-code,iso_3166-2,Sub Region,ISO Region,IMAGE Region Code,IMAGE Region Name
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,South Asia,Asia,RSAS,Rest S. Asia
1,Albania,AL,ALB,8,ISO 3166-2:AL,Europe and Russia,Europe,CEU,C. Europe
2,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Middle East,Africa,NAF,N. Africa
3,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Oceania,OCE,Oceania
4,Andorra,AD,AND,20,ISO 3166-2:AD,Europe and Russia,Europe,WEU,W. Europe
...,...,...,...,...,...,...,...,...,...
232,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Oceania,OCE,Oceania
233,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Middle East,Africa,NAF,N. Africa
234,Yemen,YE,YEM,887,ISO 3166-2:YE,Middle East,Asia,ME,Middle East
235,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,sub-Saharan Africa,Africa,RSAF,Rest S. Africa


In [30]:
ccode_iso_column = ccode_iso[['ISO Country','IMAGE Region Name']]
ccode_iso_column

,ISO Country,IMAGE Region Name
0,Afghanistan,Rest S. Asia
1,Albania,C. Europe
2,Algeria,N. Africa
3,American Samoa,Oceania
4,Andorra,W. Europe
...,...,...
232,Wallis and Futuna,Oceania
233,Western Sahara,N. Africa
234,Yemen,Middle East
235,Zambia,Rest S. Africa


In [33]:
merged_df_region = pd.merge(left=merged_df, right=ccode_iso_column, how="left", left_on="country", right_on="ISO Country")
merged_df_region

,longitude,latitude,country,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,GLCT1980,...,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020,ISO Country,IMAGE Region Name
0,-179.958333,-16.875000,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351,Fiji,Oceania
1,-179.958333,-16.791666,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351,Fiji,Oceania
2,-179.958333,-16.541666,Fiji,trof1970,2845.649902,#799351,trof1975,2845.649902,#799351,trof1980,...,2845.649902,#799351,trof2015,2845.649902,#799351,trof2020,2845.649902,#799351,Fiji,Oceania
3,-179.875000,-16.791666,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351,Fiji,Oceania
4,-179.875000,-16.708333,Fiji,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,...,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351,Fiji,Oceania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215824,152.124999,-24.791666,Australia,svna1970,7794.999512,#D10363,svna1975,7794.999512,#D10363,svna1980,...,7794.999512,#F3CA52,agri2015,7794.999512,#F3CA52,agri2020,7794.999512,#F3CA52,Australia,Oceania
2215825,152.124999,-24.708333,Australia,svna1970,7800.229492,#D10363,svna1975,7800.229492,#D10363,svna1980,...,7800.229492,#D10363,svna2015,7800.229492,#D10363,svna2020,7800.229492,#D10363,Australia,Oceania
2215826,152.124999,-24.625000,Australia,svna1970,5697.569824,#D10363,svna1975,5697.569824,#D10363,svna1980,...,5697.569824,#D10363,svna2015,5697.569824,#D10363,svna2020,5697.569824,#D10363,Australia,Oceania
2215827,152.374999,-24.708333,Australia,svna1970,606.515991,#D10363,svna1975,606.515991,#D10363,svna1980,...,606.515991,#D10363,svna2015,606.515991,#D10363,svna2020,606.515991,#D10363,Australia,Oceania


In [34]:
merged_df_region_columns = merged_df_region[['longitude', 'latitude', 'country','IMAGE Region Name',
       'GLCT1970', 'GAREA1970', 'color1970', 'GLCT1975', 'GAREA1975',
       'color1975', 'GLCT1980', 'GAREA1980', 'color1980', 'GLCT1985',
       'GAREA1985', 'color1985', 'GLCT1990', 'GAREA1990', 'color1990',
       'GLCT1995', 'GAREA1995', 'color1995', 'GLCT2000', 'GAREA2000',
       'color2000', 'GLCT2005', 'GAREA2005', 'color2005', 'GLCT2010',
       'GAREA2010', 'color2010', 'GLCT2015', 'GAREA2015', 'color2015',
       'GLCT2020', 'GAREA2020', 'color2020']]
merged_df_region_columns

,longitude,latitude,country,IMAGE Region Name,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,...,color2005,GLCT2010,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020
0,-179.958333,-16.875000,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
1,-179.958333,-16.791666,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
2,-179.958333,-16.541666,Fiji,Oceania,trof1970,2845.649902,#799351,trof1975,2845.649902,#799351,...,#799351,trof2010,2845.649902,#799351,trof2015,2845.649902,#799351,trof2020,2845.649902,#799351
3,-179.875000,-16.791666,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
4,-179.875000,-16.708333,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215824,152.124999,-24.791666,Australia,Oceania,svna1970,7794.999512,#D10363,svna1975,7794.999512,#D10363,...,#F3CA52,agri2010,7794.999512,#F3CA52,agri2015,7794.999512,#F3CA52,agri2020,7794.999512,#F3CA52
2215825,152.124999,-24.708333,Australia,Oceania,svna1970,7800.229492,#D10363,svna1975,7800.229492,#D10363,...,#D10363,svna2010,7800.229492,#D10363,svna2015,7800.229492,#D10363,svna2020,7800.229492,#D10363
2215826,152.124999,-24.625000,Australia,Oceania,svna1970,5697.569824,#D10363,svna1975,5697.569824,#D10363,...,#D10363,svna2010,5697.569824,#D10363,svna2015,5697.569824,#D10363,svna2020,5697.569824,#D10363
2215827,152.374999,-24.708333,Australia,Oceania,svna1970,606.515991,#D10363,svna1975,606.515991,#D10363,...,#D10363,svna2010,606.515991,#D10363,svna2015,606.515991,#D10363,svna2020,606.515991,#D10363


In [35]:
merged_df_columns = merged_df_region_columns.drop(columns=['longitude','latitude'])
merged_df_columns.head()

,country,IMAGE Region Name,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,GLCT1980,GAREA1980,...,color2005,GLCT2010,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020
0,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,0.000000,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
1,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,0.000000,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
2,Fiji,Oceania,trof1970,2845.649902,#799351,trof1975,2845.649902,#799351,trof1980,2845.649902,...,#799351,trof2010,2845.649902,#799351,trof2015,2845.649902,#799351,trof2020,2845.649902,#799351
3,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,0.000000,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351
4,Fiji,Oceania,trof1970,0.000000,#799351,trof1975,0.000000,#799351,trof1980,0.000000,...,#799351,trof2010,0.000000,#799351,trof2015,0.000000,#799351,trof2020,0.000000,#799351


In [36]:
merged_df_columns.to_csv("/vol/milkunarc/cadlan/stream_2/Step5/sankey_garea_finall_region.csv", index=False)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

df_sankey_garea = pd.read_csv("D:/kerja/asisten riset/code v/review/sankey_garea_finall_region.csv", low_memory=False)
df_sankey_garea.head()

,country,IMAGE Region Name,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,GLCT1980,GAREA1980,...,color2005,GLCT2010,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020
0,Fiji,Oceania,trof1970,0.00,#799351,trof1975,0.00,#799351,trof1980,0.00,...,#799351,trof2010,0.00,#799351,trof2015,0.00,#799351,trof2020,0.00,#799351
1,Fiji,Oceania,trof1970,0.00,#799351,trof1975,0.00,#799351,trof1980,0.00,...,#799351,trof2010,0.00,#799351,trof2015,0.00,#799351,trof2020,0.00,#799351
2,Fiji,Oceania,trof1970,2845.65,#799351,trof1975,2845.65,#799351,trof1980,2845.65,...,#799351,trof2010,2845.65,#799351,trof2015,2845.65,#799351,trof2020,2845.65,#799351
3,Fiji,Oceania,trof1970,0.00,#799351,trof1975,0.00,#799351,trof1980,0.00,...,#799351,trof2010,0.00,#799351,trof2015,0.00,#799351,trof2020,0.00,#799351
4,Fiji,Oceania,trof1970,0.00,#799351,trof1975,0.00,#799351,trof1980,0.00,...,#799351,trof2010,0.00,#799351,trof2015,0.00,#799351,trof2020,0.00,#799351


In [2]:
# df_sankey_garea['GLCT1970'].value_counts()

In [3]:
# df_sankey_garea['GLCT1975'].value_counts()

In [2]:
df_sankey_garea_copy = df_sankey_garea.copy()
# Assuming your dataframe is named 'df'

# List of columns to modify
glct_columns = [f"GLCT{year}" for year in range(1970, 2025, 5)]

# List of text values to replace
forested_text_to_replace  = ['rfab', 'rftm', 'tdfo', 'tmfo', 'trof', 'trow', 'wmfo', 'bore', 'ccfo']
non_forested_text_to_replace = ['dsrt', 'scrb', 'stepp', 'svna', 'tund', 'tuwd']
biofuel = ['biof']

# Replace text in specified columns
for column in glct_columns:
    df_sankey_garea_copy[column] = df_sankey_garea_copy[column].str.replace('|'.join(forested_text_to_replace), 'forest', regex=True)
    df_sankey_garea_copy[column] = df_sankey_garea_copy[column].str.replace('|'.join(non_forested_text_to_replace), 'nonforest', regex=True)
    df_sankey_garea_copy[column] = df_sankey_garea_copy[column].str.replace('|'.join(biofuel), 'agri', regex=True)

# Print the modified dataframe
df_sankey_garea_drop = df_sankey_garea_copy.drop(columns=['country'])
df_sankey_garea_drop

,IMAGE Region Name,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,GLCT1980,GAREA1980,color1980,...,color2005,GLCT2010,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020
0,Oceania,forest1970,0.0000,#799351,forest1975,0.0000,#799351,forest1980,0.0000,#799351,...,#799351,forest2010,0.0000,#799351,forest2015,0.0000,#799351,forest2020,0.0000,#799351
1,Oceania,forest1970,0.0000,#799351,forest1975,0.0000,#799351,forest1980,0.0000,#799351,...,#799351,forest2010,0.0000,#799351,forest2015,0.0000,#799351,forest2020,0.0000,#799351
2,Oceania,forest1970,2845.6500,#799351,forest1975,2845.6500,#799351,forest1980,2845.6500,#799351,...,#799351,forest2010,2845.6500,#799351,forest2015,2845.6500,#799351,forest2020,2845.6500,#799351
3,Oceania,forest1970,0.0000,#799351,forest1975,0.0000,#799351,forest1980,0.0000,#799351,...,#799351,forest2010,0.0000,#799351,forest2015,0.0000,#799351,forest2020,0.0000,#799351
4,Oceania,forest1970,0.0000,#799351,forest1975,0.0000,#799351,forest1980,0.0000,#799351,...,#799351,forest2010,0.0000,#799351,forest2015,0.0000,#799351,forest2020,0.0000,#799351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215824,Oceania,nonforest1970,7794.9995,#D10363,nonforest1975,7794.9995,#D10363,nonforest1980,7794.9995,#D10363,...,#F3CA52,agri2010,7794.9995,#F3CA52,agri2015,7794.9995,#F3CA52,agri2020,7794.9995,#F3CA52
2215825,Oceania,nonforest1970,7800.2295,#D10363,nonforest1975,7800.2295,#D10363,nonforest1980,7800.2295,#D10363,...,#D10363,nonforest2010,7800.2295,#D10363,nonforest2015,7800.2295,#D10363,nonforest2020,7800.2295,#D10363
2215826,Oceania,nonforest1970,5697.5700,#D10363,nonforest1975,5697.5700,#D10363,nonforest1980,5697.5700,#D10363,...,#D10363,nonforest2010,5697.5700,#D10363,nonforest2015,5697.5700,#D10363,nonforest2020,5697.5700,#D10363
2215827,Oceania,nonforest1970,606.5160,#D10363,nonforest1975,606.5160,#D10363,nonforest1980,606.5160,#D10363,...,#D10363,nonforest2010,606.5160,#D10363,nonforest2015,606.5160,#D10363,nonforest2020,606.5160,#D10363


In [3]:
for tahun in range(1970, 2021, 5):
    color_mapping = {
        f'agri{tahun}': '#FF6600',
        f'forest{tahun}': '#347928',
        f'nonforest{tahun}': '#B7E0FF',
        f'grass{tahun}': '#FCCD2A'
    }
    df_sankey_garea_drop.loc[df_sankey_garea_drop[f'GLCT{tahun}'].isin(color_mapping.keys()), f'color{tahun}'] = df_sankey_garea_drop[f'GLCT{tahun}'].map(color_mapping)

df_sankey_garea_copy_nan = df_sankey_garea_drop.dropna()
df_sankey_garea_copy_nan

,IMAGE Region Name,GLCT1970,GAREA1970,color1970,GLCT1975,GAREA1975,color1975,GLCT1980,GAREA1980,color1980,...,color2005,GLCT2010,GAREA2010,color2010,GLCT2015,GAREA2015,color2015,GLCT2020,GAREA2020,color2020
0,Oceania,forest1970,0.0000,#347928,forest1975,0.0000,#347928,forest1980,0.0000,#347928,...,#347928,forest2010,0.0000,#347928,forest2015,0.0000,#347928,forest2020,0.0000,#347928
1,Oceania,forest1970,0.0000,#347928,forest1975,0.0000,#347928,forest1980,0.0000,#347928,...,#347928,forest2010,0.0000,#347928,forest2015,0.0000,#347928,forest2020,0.0000,#347928
2,Oceania,forest1970,2845.6500,#347928,forest1975,2845.6500,#347928,forest1980,2845.6500,#347928,...,#347928,forest2010,2845.6500,#347928,forest2015,2845.6500,#347928,forest2020,2845.6500,#347928
3,Oceania,forest1970,0.0000,#347928,forest1975,0.0000,#347928,forest1980,0.0000,#347928,...,#347928,forest2010,0.0000,#347928,forest2015,0.0000,#347928,forest2020,0.0000,#347928
4,Oceania,forest1970,0.0000,#347928,forest1975,0.0000,#347928,forest1980,0.0000,#347928,...,#347928,forest2010,0.0000,#347928,forest2015,0.0000,#347928,forest2020,0.0000,#347928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215824,Oceania,nonforest1970,7794.9995,#B7E0FF,nonforest1975,7794.9995,#B7E0FF,nonforest1980,7794.9995,#B7E0FF,...,#FF6600,agri2010,7794.9995,#FF6600,agri2015,7794.9995,#FF6600,agri2020,7794.9995,#FF6600
2215825,Oceania,nonforest1970,7800.2295,#B7E0FF,nonforest1975,7800.2295,#B7E0FF,nonforest1980,7800.2295,#B7E0FF,...,#B7E0FF,nonforest2010,7800.2295,#B7E0FF,nonforest2015,7800.2295,#B7E0FF,nonforest2020,7800.2295,#B7E0FF
2215826,Oceania,nonforest1970,5697.5700,#B7E0FF,nonforest1975,5697.5700,#B7E0FF,nonforest1980,5697.5700,#B7E0FF,...,#B7E0FF,nonforest2010,5697.5700,#B7E0FF,nonforest2015,5697.5700,#B7E0FF,nonforest2020,5697.5700,#B7E0FF
2215827,Oceania,nonforest1970,606.5160,#B7E0FF,nonforest1975,606.5160,#B7E0FF,nonforest1980,606.5160,#B7E0FF,...,#B7E0FF,nonforest2010,606.5160,#B7E0FF,nonforest2015,606.5160,#B7E0FF,nonforest2020,606.5160,#B7E0FF


df_sankey_garea_copy_nan = df_sankey_garea_copy[df_sankey_garea_copy['IMAGE Region Name'].isna()]
df_sankey_garea_copy_nan

In [13]:
tahun = [1970, 2020]

columns_to_select = []
columns_groupby = []

columns_to_select.append("IMAGE Region Name")
columns_groupby.append("IMAGE Region Name")

df_copy1 = df_sankey_garea_copy_nan.copy()
for t in tahun:
    columns_to_select.extend([f'GLCT{t}', f'GAREA{t}', f'color{t}'])
    columns_groupby.extend([f'GLCT{t}', f'color{t}'])

df_selected1 = df_copy1[columns_to_select]
df_selected1

,IMAGE Region Name,GLCT1970,GAREA1970,color1970,GLCT2020,GAREA2020,color2020
0,Oceania,forest1970,0.0000,#347928,forest2020,0.0000,#347928
1,Oceania,forest1970,0.0000,#347928,forest2020,0.0000,#347928
2,Oceania,forest1970,2845.6500,#347928,forest2020,2845.6500,#347928
3,Oceania,forest1970,0.0000,#347928,forest2020,0.0000,#347928
4,Oceania,forest1970,0.0000,#347928,forest2020,0.0000,#347928
...,...,...,...,...,...,...,...
2215824,Oceania,nonforest1970,7794.9995,#B7E0FF,agri2020,7794.9995,#FF6600
2215825,Oceania,nonforest1970,7800.2295,#B7E0FF,nonforest2020,7800.2295,#B7E0FF
2215826,Oceania,nonforest1970,5697.5700,#B7E0FF,nonforest2020,5697.5700,#B7E0FF
2215827,Oceania,nonforest1970,606.5160,#B7E0FF,nonforest2020,606.5160,#B7E0FF


In [14]:
df_selected_group1 = df_selected1.groupby(columns_groupby).sum()
df_selected_index1 = df_selected_group1.reset_index()
df_selected_index1

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
0,Brazil,agri1970,#FF6600,agri2020,#FF6600,1.953410e+08,1.938697e+08
1,Brazil,forest1970,#347928,agri2020,#FF6600,3.891960e+07,3.867473e+07
2,Brazil,forest1970,#347928,forest2020,#347928,4.160692e+08,4.154912e+08
3,Brazil,grass1970,#FCCD2A,forest2020,#347928,6.127858e+04,5.818165e+04
4,Brazil,grass1970,#FCCD2A,nonforest2020,#B7E0FF,1.518636e+05,1.513886e+05
...,...,...,...,...,...,...,...
225,W. Europe,grass1970,#FCCD2A,forest2020,#347928,4.339161e+04,4.275635e+04
226,W. Europe,grass1970,#FCCD2A,grass2020,#FCCD2A,2.025457e+05,2.029769e+05
227,W. Europe,grass1970,#FCCD2A,nonforest2020,#B7E0FF,8.464599e+04,8.431411e+04
228,W. Europe,nonforest1970,#B7E0FF,agri2020,#FF6600,7.885791e+05,7.720949e+05


In [15]:
df_filtered1 = df_selected_index1[~df_selected_index1.apply(lambda row: row.astype(str).str.contains('ice').any(), axis=1)]
df_filtered1

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
0,Brazil,agri1970,#FF6600,agri2020,#FF6600,1.953410e+08,1.938697e+08
1,Brazil,forest1970,#347928,agri2020,#FF6600,3.891960e+07,3.867473e+07
2,Brazil,forest1970,#347928,forest2020,#347928,4.160692e+08,4.154912e+08
3,Brazil,grass1970,#FCCD2A,forest2020,#347928,6.127858e+04,5.818165e+04
4,Brazil,grass1970,#FCCD2A,nonforest2020,#B7E0FF,1.518636e+05,1.513886e+05
...,...,...,...,...,...,...,...
225,W. Europe,grass1970,#FCCD2A,forest2020,#347928,4.339161e+04,4.275635e+04
226,W. Europe,grass1970,#FCCD2A,grass2020,#FCCD2A,2.025457e+05,2.029769e+05
227,W. Europe,grass1970,#FCCD2A,nonforest2020,#B7E0FF,8.464599e+04,8.431411e+04
228,W. Europe,nonforest1970,#B7E0FF,agri2020,#FF6600,7.885791e+05,7.720949e+05


In [54]:
new_dataA = pd.read_excel("D:/kerja/asisten riset/code v/review/panelA_clean.xlsx")
new_dataA.head()

,IMAGE Region Name,forest,panel_sum
0,Brazil,forest,3.767912e+07
1,Brazil,nonforest,5.672303e+07
2,C. Europe,forest,4.452729e+05
3,C. Europe,nonforest,1.278727e+04
4,Canada,forest,1.231777e+06


In [55]:
panelA = df_filtered1[(df_filtered1['GLCT1970'] != 'agri1970') & (df_filtered1['GLCT2020'] == 'agri2020')]
panelA.loc[:, 'GAREA1970'] = 0
panelA.loc[:, 'GAREA2020'] = 0
panelA

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
1,Brazil,forest1970,#347928,agri2020,#FF6600,0.0,0.0
5,Brazil,nonforest1970,#B7E0FF,agri2020,#FF6600,0.0,0.0
11,C. Europe,forest1970,#347928,agri2020,#FF6600,0.0,0.0
13,C. Europe,nonforest1970,#B7E0FF,agri2020,#FF6600,0.0,0.0
18,Canada,forest1970,#347928,agri2020,#FF6600,0.0,0.0
21,Canada,nonforest1970,#B7E0FF,agri2020,#FF6600,0.0,0.0
25,China,forest1970,#347928,agri2020,#FF6600,0.0,0.0
31,China,nonforest1970,#B7E0FF,agri2020,#FF6600,0.0,0.0
36,E. Africa,forest1970,#347928,agri2020,#FF6600,0.0,0.0
40,E. Africa,nonforest1970,#B7E0FF,agri2020,#FF6600,0.0,0.0


In [69]:

# jika pada data `panelA` memiliki IMAGE Region Name yang sama dengan IMAGE Region Name pada data `new_dataA`, dan memiliki `GLCT1970` yang sama pada data `panelA` dengan `forest` pada data `new_dataA`
# contoh kolom `GLCT1970` pada data `panelA` berisi `forest1970` panelA[panelA.apply(lambda row: row.astype(str).str.contains('forest').any(), axis=1)], dan kolom  `forest` berisi `forest` pada data `new_dataA`

def update_panel_data(panel, new_data, data):
    """
    Update panel data based on matching conditions with new data.
    
    Parameters:
    panel : pandas DataFrame
        Panel data with GLCT1970, GLCT2020, and GAREA columns
    new_data : pandas DataFrame
        New data with forest type information and values
        
    Returns:
    pandas DataFrame
        Updated panel data with merged area values
    """
    if data == "A":
        # Create a copy of panel to avoid modifying original data
        updated_panel = panel.copy()
        
        # Create mapping conditions
        def map_values(row):
            # Find matching rows in new_data
            matching_row = new_data[
                (new_data['IMAGE Region Name'] == row['IMAGE Region Name']) &
                (new_data['forest'] == row['GLCT1970'].replace('1970', ''))
            ]
            
            if not matching_row.empty:
                # Update 1970 and 2020 values if match found
                row['GAREA1970'] = matching_row['panel_sum'].values[0]
                row['GAREA2020'] = matching_row['panel_sum'].values[0]
                
            return row
        
        # Apply the mapping
        updated_panel = updated_panel.apply(map_values, axis=1)
        
        return updated_panel
    
    elif data == "B":
        # Create a copy of panel to avoid modifying original data
        updated_panel = panel.copy()
        
        # Create mapping conditions
        def map_values(row):
            # Find matching rows in new_data
            matching_row = new_data[
                (new_data['IMAGE Region Name'] == row['IMAGE Region Name']) &
                (new_data['forest'] == row['GLCT2020'].replace('2020', ''))
            ]
            
            if not matching_row.empty:
                # Update 1970 and 2020 values if match found
                row['GAREA1970'] = matching_row['panel_sum'].values[0]
                row['GAREA2020'] = matching_row['panel_sum'].values[0]
                
            return row
        
        # Apply the mapping
        updated_panel = updated_panel.apply(map_values, axis=1)
        
        return updated_panel


In [70]:
# Update values based on new_dataA
result_panelA = update_panel_data(panelA, new_dataA, data="A")
result_panelA

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
1,Brazil,forest1970,#347928,agri2020,#FF6600,3.767912e+07,3.767912e+07
5,Brazil,nonforest1970,#B7E0FF,agri2020,#FF6600,5.672303e+07,5.672303e+07
11,C. Europe,forest1970,#347928,agri2020,#FF6600,4.452729e+05,4.452729e+05
13,C. Europe,nonforest1970,#B7E0FF,agri2020,#FF6600,1.278727e+04,1.278727e+04
18,Canada,forest1970,#347928,agri2020,#FF6600,1.231777e+06,1.231777e+06
21,Canada,nonforest1970,#B7E0FF,agri2020,#FF6600,1.679671e+06,1.679671e+06
25,China,forest1970,#347928,agri2020,#FF6600,9.075342e+07,9.075342e+07
31,China,nonforest1970,#B7E0FF,agri2020,#FF6600,6.737439e+07,6.737439e+07
36,E. Africa,forest1970,#347928,agri2020,#FF6600,9.719229e+06,9.719229e+06
40,E. Africa,nonforest1970,#B7E0FF,agri2020,#FF6600,3.826849e+07,3.826849e+07


In [67]:
new_dataB = pd.read_excel("D:/kerja/asisten riset/code v/review/panelB_clean.xlsx")
new_dataB.head()

,IMAGE Region Name,forest,panel_sum
0,C. Europe,forest,2.235258e+06
1,Middle East,nonforest,3.109413e+06
2,Oceania,nonforest,2.877151e+07
3,Russia,nonforest,5.140468e+06
4,Turkey,nonforest,4.072691e+06


In [68]:
panelB = df_filtered1[(df_filtered1['GLCT1970'] == 'agri1970') & (df_filtered1['GLCT2020'] != 'agri2020')]
panelB = panelB[(panelB['GLCT1970'] == 'agri1970') &  (panelB['GLCT2020'] != 'grass2020')]
panelB.loc[:, 'GAREA1970'] = 0
panelB.loc[:, 'GAREA2020'] = 0
panelB

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
9,C. Europe,agri1970,#FF6600,forest2020,#347928,0.0,0.0
10,C. Europe,agri1970,#FF6600,nonforest2020,#B7E0FF,0.0,0.0
16,Canada,agri1970,#FF6600,forest2020,#347928,0.0,0.0
17,Canada,agri1970,#FF6600,nonforest2020,#B7E0FF,0.0,0.0
24,China,agri1970,#FF6600,nonforest2020,#B7E0FF,0.0,0.0
35,E. Africa,agri1970,#FF6600,nonforest2020,#B7E0FF,0.0,0.0
44,India,agri1970,#FF6600,forest2020,#347928,0.0,0.0
45,India,agri1970,#FF6600,nonforest2020,#B7E0FF,0.0,0.0
53,Indonesia,agri1970,#FF6600,forest2020,#347928,0.0,0.0
54,Indonesia,agri1970,#FF6600,nonforest2020,#B7E0FF,0.0,0.0


In [71]:
# Update values based on new_dataA
result_panelB = update_panel_data(panelB, new_dataB, data="B")
result_panelB

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
9,C. Europe,agri1970,#FF6600,forest2020,#347928,2.235258e+06,2.235258e+06
10,C. Europe,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00
16,Canada,agri1970,#FF6600,forest2020,#347928,0.000000e+00,0.000000e+00
17,Canada,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00
24,China,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00
35,E. Africa,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00
44,India,agri1970,#FF6600,forest2020,#347928,0.000000e+00,0.000000e+00
45,India,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00
53,Indonesia,agri1970,#FF6600,forest2020,#347928,0.000000e+00,0.000000e+00
54,Indonesia,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00


In [72]:
result_drop_overtime = pd.concat([result_panelA, result_panelB], ignore_index=True)

In [92]:
result_drop_overtime[result_drop_overtime['IMAGE Region Name'] == "Indonesia"]

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020
12,Indonesia,forest1970,#347928,agri2020,#FF6600,2.285258e+07,2.285258e+07
13,Indonesia,nonforest1970,#B7E0FF,agri2020,#FF6600,1.329301e+06,1.329301e+06
56,Indonesia,agri1970,#FF6600,forest2020,#347928,0.000000e+00,0.000000e+00
57,Indonesia,agri1970,#FF6600,nonforest2020,#B7E0FF,0.000000e+00,0.000000e+00


In [93]:
# // v1/(v1 + v2) * 100 
# // v2/(v1 + v2) * 100
# // v3/(v3 + v4) * 100
# // v4/(v3 + v4) * 100

import pandas as pd
import numpy as np

def calculate_percentage(df, year_col='GAREA2020', group_size=2):
    """
    Calculate percentages for pairs of rows based on a specified column.
    
    Parameters:
    df : pandas DataFrame
        Input DataFrame
    year_col : str
        Column name to use for percentage calculations
    group_size : int
        Size of groups to calculate percentages (default is 2 for pairs)
        
    Returns:
    pandas DataFrame
        DataFrame with added 'percent' column
    """
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Calculate group sums
    group_indices = np.arange(len(df)) // group_size
    group_sums = df.groupby(group_indices)[year_col].transform('sum')
    
    # Calculate percentages
    df['percent'] = (df[year_col] / group_sums * 100).round(2)
    
    return df

# Example usage:
result = calculate_percentage(result_drop_overtime)
result = result.dropna()
result = result[(result['GAREA1970'] != 0) & (result['GAREA2020'] != 0)]

result[result['IMAGE Region Name'] == "Indonesia"]

,IMAGE Region Name,GLCT1970,color1970,GLCT2020,color2020,GAREA1970,GAREA2020,percent
12,Indonesia,forest1970,#347928,agri2020,#FF6600,2.285258e+07,2.285258e+07,94.5
13,Indonesia,nonforest1970,#B7E0FF,agri2020,#FF6600,1.329301e+06,1.329301e+06,5.5


In [101]:
import pandas as pd

def split_by_region(df, region_col='IMAGE Region Name'):
    """
    Split DataFrame into separate DataFrames by region.
    
    Parameters:
    df : pandas DataFrame
        Input DataFrame to be split
    region_col : str
        Name of the region column
        
    Returns:
    dict : Dictionary of DataFrames where keys are region names
    """
    # Split data into dictionary of DataFrames
    region_dfs = {
        region: group.copy() for region, group in df.groupby(region_col)
    }
    
    return region_dfs

def display_region_data(region_dfs):
    """
    Display data for each region in a formatted way.
    
    Parameters:
    region_dfs : dict
        Dictionary of DataFrames split by region
    """
    for region, df in region_dfs.items():
        print(f"\n{'='*50}")
        print(f"Region: {region}")
        print(f"{'='*50}")
        print(df.to_string(index=False))
        print(f"\nShape: {df.shape}")
        print(f"Total GAREA1970: {df['GAREA1970'].sum():,.2f}")
        print(f"Total GAREA2020: {df['GAREA2020'].sum():,.2f}")

# Split data by region
region_data = split_by_region(result)


# Access individual region data
# Indonesia_data = region_data['Indonesia']
# china_data = region_data['China']


# Display all region data
# display_region_data(region_data)

# Save individual region data to CSV (optional)
# for region, df in region_data.items():
    # df.to_csv(f"{region.lower().replace(' ', '_')}_data.csv", index=False)


In [ ]:
import json
import pandas as pd

def generate_sankey_data(df, years, glct_prefix='GLCT', garea_prefix='GAREA'):
    """
    Generate Sankey diagram data from a DataFrame with percentages in nodes.
    
    Parameters:
    df : pandas DataFrame
        Input DataFrame containing GLCT and GAREA columns
    years : list
        List of years to process
    glct_prefix : str
        Prefix for GLCT columns
    garea_prefix : str
        Prefix for GAREA columns
    
    Returns:
    dict : Sankey diagram data structure
    """
    # Collect unique nodes and their percentages from all years
    nodes = {}  # Dictionary to store node information including percentages
    for year in years:
        col_name = f"{glct_prefix}{year}"
        # Group by GLCT and calculate percentage for each node
        year_data = df.groupby(col_name).agg({
            'percent': 'first'  # Mengambil nilai percent pertama untuk setiap node
        }).reset_index()
        
        for _, row in year_data.iterrows():
            node_name = row[col_name]
            if node_name not in nodes:
                # Format percent as string with % symbol
                percent_str = f"{float(row['percent']):.0f}%" if 'percent' in row else None
                nodes[node_name] = {
                    'name': node_name,
                    'percent': percent_str
                }

    # Create node dictionary with indices and create final nodes list
    nodes_dict = {}  # For mapping node names to indices
    nodes_data = []  # Final list of node objects
    
    for index, (node_name, node_info) in enumerate(nodes.items()):
        nodes_dict[node_name] = index
        node_data = {
            "node": index,
            "name": node_name,
            "percent": node_info['percent']
        }
        nodes_data.append(node_data)

    # Generate links between consecutive years
    links = []
    for i in range(len(years) - 1):
        current_year = years[i]
        next_year = years[i + 1]
        
        # Process each row in the DataFrame
        for _, row in df.iterrows():
            source = nodes_dict[row[f'{glct_prefix}{current_year}']]
            target = nodes_dict[row[f'{glct_prefix}{next_year}']]
            value = float(row[f'{garea_prefix}{current_year}'])
            
            # Only add link if source and target are different
            if source != target:
                link = {
                    "source": source,
                    "target": target,
                    "value": value
                }
                links.append(link)

    # Create final Sankey data structure
    sankey_data = {
        "nodes": nodes_data,
        "links": links
    }
    
    return sankey_data

In [102]:
# Define years
years = [1970, 2020]

for region in result['IMAGE Region Name'].unique():

    sankey_data = generate_sankey_data(region_data[region], years)
    
    # Save to JSON file
    with open(f'D:/kerja/asisten riset/code v/review/data_json_new/sankey_data_{region}.json', 'w') as json_file:
        json.dump(sankey_data, json_file, indent=4)

In [27]:
import plotly.graph_objects as go

nodes = {}
links = []
node_colors = []
link_colors = []

for i in range(len(tahun) - 1):
    current_year = tahun[i]
    next_year = tahun[i + 1]

    gltct_current = f'GLCT{current_year}'
    color_current = f'color{current_year}'
    value_current = f'GAREA{current_year}'

    gltct_next = f'GLCT{next_year}'
    color_next = f'color{next_year}'
    value_next = f'GAREA{next_year}'

    for index, row in result_drop_overtime.iterrows():
        source_node = row[gltct_current]
        target_node = row[gltct_next]
        color_source = row[color_current]

        if source_node not in nodes:
            nodes[source_node] = len(nodes)
            node_colors.append(color_source)
        if target_node not in nodes:
            nodes[target_node] = len(nodes)
            node_colors.append(row[color_next])

        link = {
            'source': nodes[source_node],
            'target': nodes[target_node],
            'value': row[value_next]
        }
        
        links.append(link)
        link_colors.append(color_source)

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=10,
        thickness=30,
        line=dict(color='white'),
        label=list(nodes.keys()),
        color=node_colors
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=link_colors,
    
    )
)])

custom_labels = ['Agriculture', 'Forested Natural Vegetation', 'Non-Forested Natural Vegetation']

nodes_1970 = [key for key in nodes.keys() if "1970" in key]
nodes_2020 = [key for key in nodes.keys() if "2020" in key]

label = ['Forested Natural Vegetation', 'Non-Forested Natural Vegetation','Agriculture', ]
warna = ['#347928','#B7E0FF','#FF6600']

legend_entries = []
for color, node in zip(warna, label):
    legend_entries.append([color, node])

legend = []
for i, entry in enumerate(legend_entries):
    legend.append(
        go.Scatter(
            mode="markers",
            x=[None],
            y=[None],
            marker=dict(size=10, color=entry[0], symbol="square"),
            name=entry[1],
            showlegend=True
        )
    )

# Create traces
traces = [fig.data[0]] + legend

# Define the layout with annotations included
layout = go.Layout(
    showlegend=True,
    plot_bgcolor="rgba(0,0,0,0)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    ),
    annotations=[
        # Simplified custom text
        dict(
            text="Cropland Gain",
            x=0.2,  # Centered for visibility
            y=1.1,    # Adjust y to ensure it's within bounds
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=16, color="black",family="Arial Black")
        ),
        dict(
            text="Cropland Loss",
            x=0.5,
            y=1.1,  # Adjusted for clarity
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=16, color="black",family="Arial Black")
        )
    ]
)

# Update the figure with traces and layout
fig = go.Figure(data=traces, layout=layout)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    width=1000,
    height=600
)

# Finally show the figure
fig.show()

In [ ]:
import plotly.graph_objects as go

# Assuming result_drop is your DataFrame with the necessary data

nodes = {}
links = []
node_colors = []
link_colors = []

# Create a mapping of nodes to their GAREA values
node_labels = {}

for i in range(len(tahun) - 1):
    current_year = tahun[i]
    next_year = tahun[i + 1]

    gltct_current = f'GLCT{current_year}'
    color_current = f'color{current_year}'
    value_current = f'GAREA{current_year}'

    gltct_next = f'GLCT{next_year}'

    for index, row in result_drop_overtime.iterrows():
        source_node = row[gltct_current]
        target_node = row[gltct_next]
        
        # Assign GAREA values as node labels
        node_labels[source_node] = row[value_current]
        node_labels[target_node] = row[f'GAREA{next_year}']

        # Assume color_source is a valid color code from your data
        color_source = row[color_current]  # Make sure this is a valid color format

        if source_node not in nodes:
            nodes[source_node] = len(nodes)
            node_colors.append(color_source)
        if target_node not in nodes:
            nodes[target_node] = len(nodes)
            node_colors.append(row[f'color{next_year}'])  # Ensure this is also a valid color format

        link = {
            'source': nodes[source_node],
            'target': nodes[target_node],
            'value': row[f'GAREA{next_year}']  # Use the correct value
        }
        
        links.append(link)
        link_colors.append(color_source)

# Prepare to create the Sankey figure
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=10,
        thickness=20,
        line=dict(color='white'),
        label=[node_labels[key] for key in nodes.keys()],  # Use GAREA values as labels
        color=node_colors  # Use your predefined colors here
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=link_colors,
    )
)])

# Create traces for the legend
custom_labels = [ 'Non-Forested Natural Vegetation', 'Forested Natural Vegetation','Agriculture']
warna = ['#B7E0FF', '#347928', '#FF6600']
legend_entries = [[color, label] for color, label in zip(warna, custom_labels)]

legend = []
for i, entry in enumerate(legend_entries):
    legend.append(
        go.Scatter(
            mode="markers",
            x=[None],
            y=[None],
            marker=dict(size=10, color=entry[0], symbol="square"),
            name=entry[1],
            showlegend=True
        )
    )

# Define the layout with annotations included
layout = go.Layout(
    showlegend=True,
    plot_bgcolor="rgba(0,0,0,0)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.45
    ),
    annotations=[
        dict(
            text="Agricultural Land Gain",
            x=0.2,
            y=1.25,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=16, color="black", family="Arial Black")
        ),
         dict(
            text="676 Mha",
            x=0.405,
            y=1.15,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color="Grey", family="Arial Black")
        ),
        dict(
            text="Agricultural Land Loss",
            x=0.6,
            y=1.25,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=16, color="black", family="Arial Black")
        ),
        dict(
            text="76 Mha",
            x=0.515,
            y=1.15,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color="Grey", family="Arial Black")
        ),
        dict(
            text="59%",
            x=0.1,
            y=0.8,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color="Black", family="Arial Black")
        )
        ,
        dict(
            text="41%",
            x=0.1,
            y=0.35,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color="Black", family="Arial Black")
        ),
        dict(
            text="84%",
            x=0.80,
            y=0.9,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color="Black", family="Arial Black")
        ),
        dict(
            text="16%",
            x=0.80,
            y=0.75,
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color="Black", family="Arial Black")
        )
    ]
)

# Update the figure with traces and layout
fig = go.Figure(data=[fig.data[0]] + legend, layout=layout)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    width=1000,
    height=400
)

# Finally show the figure
fig.show()

In [ ]:
# import json

# tahun = [1970, 2020]

# regions = df_sankey_garea_copy_nan['IMAGE Region Name'].unique()

# for region in regions:
#     df_region = df_sankey_garea_copy_nan[df_sankey_garea_copy_nan['IMAGE Region Name'] == region]

#     columns_to_select = []
#     columns_groupby = []

#     for t in tahun:
#         columns_to_select.extend([f'GLCT{t}', f'GAREA{t}', f'color{t}'])
#         columns_groupby.extend([f'GLCT{t}', f'color{t}'])

#     df_copy = df_region.copy()
#     df_selected = df_copy[columns_to_select]

#     df_selected_group = df_selected.groupby(columns_groupby).sum()
#     df_selected_index = df_selected_group.reset_index()
#     df_filtered = df_selected_index[~df_selected_index.apply(lambda row: row.astype(str).str.contains('ice').any(), axis=1)]

#     panel = df_filtered[(df_filtered['GLCT1970'] != 'agri1970') & (df_filtered['GLCT2020'] == 'agri2020')]

#     # Kumpulkan semua node (nilai unik dari kolom GLCT setiap tahun)
#     nodes = []

#     for year_col in tahun:
#         unique_values = panel[f"GLCT{year_col}"].unique()
#         for val in unique_values:
#             if val not in nodes:
#                 nodes.append(val)

#     # Membuat dictionary untuk menghubungkan node dengan indeksnya
#     nodes_dict = {node: index for index, node in enumerate(nodes)}

#     # Membuat links berdasarkan pasangan tahun berturut-turut
#     links = []

#     for i in range(len(tahun) - 1):
#         current_year = tahun[i]
#         next_year = tahun[i + 1]

#         # Loop melalui setiap baris data
#         for _, row in panel.iterrows():
#             source = nodes_dict[row[f'GLCT{current_year}']]
#             target = nodes_dict[row[f'GLCT{next_year}']]
#             value = row[f'GAREA{current_year}']  # Anda bisa sesuaikan value dengan logika yang dibutuhkan
            
#             # Hanya tambahkan link jika source dan target berbeda
#             if source != target:
#                 links.append({
#                     "source": source,
#                     "target": target,
#                     "value": value
#                 })

#     # Membuat data yang akan digunakan untuk Sankey diagram
#     sankey_data = {
#         "nodes": [{"node": index, "name": name} for name, index in nodes_dict.items()],
#         "links": links
#     }

#     # Simpan ke file JSON
#     with open(f'data_json/sankey_data_{region}_panelA.json', 'w') as json_file:
#         json.dump(sankey_data, json_file, indent=4)


In [ ]:
# import json

# tahun = [1970, 2020]

# regions = df_sankey_garea_copy_nan['IMAGE Region Name'].unique()

# for region in regions:
#     df_region = df_sankey_garea_copy_nan[df_sankey_garea_copy_nan['IMAGE Region Name'] == region]

#     columns_to_select = []
#     columns_groupby = []

#     for t in tahun:
#         columns_to_select.extend([f'GLCT{t}', f'GAREA{t}', f'color{t}'])
#         columns_groupby.extend([f'GLCT{t}', f'color{t}'])

#     df_copy = df_region.copy()
#     df_selected = df_copy[columns_to_select]

#     df_selected_group = df_selected.groupby(columns_groupby).sum()
#     df_selected_index = df_selected_group.reset_index()
#     df_filtered = df_selected_index[~df_selected_index.apply(lambda row: row.astype(str).str.contains('ice').any(), axis=1)]

#     panel = df_filtered[(df_filtered['GLCT1970'] == 'agri1970') & (df_filtered['GLCT2020'] != 'agri2020')]

#     # Kumpulkan semua node (nilai unik dari kolom GLCT setiap tahun)
#     nodes = []

#     for year_col in tahun:
#         unique_values = panel[f"GLCT{year_col}"].unique()
#         for val in unique_values:
#             if val not in nodes:
#                 nodes.append(val)

#     # Membuat dictionary untuk menghubungkan node dengan indeksnya
#     nodes_dict = {node: index for index, node in enumerate(nodes)}

#     # Membuat links berdasarkan pasangan tahun berturut-turut
#     links = []

#     for i in range(len(tahun) - 1):
#         current_year = tahun[i]
#         next_year = tahun[i + 1]

#         # Loop melalui setiap baris data
#         for _, row in panel.iterrows():
#             source = nodes_dict[row[f'GLCT{current_year}']]
#             target = nodes_dict[row[f'GLCT{next_year}']]
#             value = row[f'GAREA{current_year}']  # Anda bisa sesuaikan value dengan logika yang dibutuhkan
            
#             # Hanya tambahkan link jika source dan target berbeda
#             if source != target:
#                 links.append({
#                     "source": source,
#                     "target": target,
#                     "value": value
#                 })

#     # Membuat data yang akan digunakan untuk Sankey diagram
#     sankey_data = {
#         "nodes": [{"node": index, "name": name} for name, index in nodes_dict.items()],
#         "links": links
#     }

#     # Simpan ke file JSON
#     with open(f'data_json/sankey_data_{region}_panelB.json', 'w') as json_file:
#         json.dump(sankey_data, json_file, indent=4)


In [66]:
# import json

# # Daftar tahun yang dinamis
# tahun = [1970, 2020]

# # Kumpulkan semua node (nilai unik dari kolom GLCT setiap tahun)
# nodes = []
# for year_col in tahun:
#     unique_values = result_drop_overtime[f"GLCT{year_col}"].unique()
#     for val in unique_values:
#         if val not in nodes:
#             nodes.append(val)

# # Membuat dictionary untuk menghubungkan node dengan indeksnya
# nodes_dict = {node: index for index, node in enumerate(nodes)}

# # Membuat links berdasarkan pasangan tahun berturut-turut
# links = []

# for i in range(len(tahun) - 1):
#     current_year = tahun[i]
#     next_year = tahun[i + 1]

#     # Loop melalui setiap baris data
#     for _, row in result_drop_overtime.iterrows():
#         source = nodes_dict[row[f'GLCT{current_year}']]
#         target = nodes_dict[row[f'GLCT{next_year}']]
#         value = row[f'GAREA{current_year}']  # Anda bisa sesuaikan value dengan logika yang dibutuhkan
#         # nodes_dict.append({"percent": row['percent']}) # menambahkan key dan value percent

        
#         # Hanya tambahkan link jika source dan target berbeda
#         if source != target:
#             link = {
#                 "source": source,
#                 "target": target,
#                 "value": value
#             }

#             if 'percent' in row:
#                 link["percent"] = float(row['percent'])
            
#             links.append(link)

# # Membuat data yang akan digunakan untuk Sankey diagram
# sankey_data = {
#     "nodes": [{"node": index, "name": name} for index, name in nodes_dict.items()],
#     "links": links
# }
# sankey_data

# # # Simpan ke file JSON
# # with open('sankey_data_result_drop_overtime.json', 'w') as json_file:
# #     json.dump(sankey_data, json_file, indent=4)


In [83]:
import json
import pandas as pd

def generate_sankey_data(df, years, glct_prefix='GLCT', garea_prefix='GAREA'):
    """
    Generate Sankey diagram data from a DataFrame with percentages in nodes.
    
    Parameters:
    df : pandas DataFrame
        Input DataFrame containing GLCT and GAREA columns
    years : list
        List of years to process
    glct_prefix : str
        Prefix for GLCT columns
    garea_prefix : str
        Prefix for GAREA columns
    
    Returns:
    dict : Sankey diagram data structure
    """
    # Collect unique nodes and their percentages from all years
    nodes = {}  # Dictionary to store node information including percentages
    for year in years:
        col_name = f"{glct_prefix}{year}"
        # Group by GLCT and calculate percentage for each node
        year_data = df.groupby(col_name).agg({
            'percent': 'first'  # Mengambil nilai percent pertama untuk setiap node
        }).reset_index()
        
        for _, row in year_data.iterrows():
            node_name = row[col_name]
            if node_name not in nodes:
                # Format percent as string with % symbol
                percent_str = f"{float(row['percent']):.0f}%" if 'percent' in row else None
                nodes[node_name] = {
                    'name': node_name,
                    'percent': percent_str
                }

    # Create node dictionary with indices and create final nodes list
    nodes_dict = {}  # For mapping node names to indices
    nodes_data = []  # Final list of node objects
    
    for index, (node_name, node_info) in enumerate(nodes.items()):
        nodes_dict[node_name] = index
        node_data = {
            "node": index,
            "name": node_name,
            "percent": node_info['percent']
        }
        nodes_data.append(node_data)

    # Generate links between consecutive years
    links = []
    for i in range(len(years) - 1):
        current_year = years[i]
        next_year = years[i + 1]
        
        # Process each row in the DataFrame
        for _, row in df.iterrows():
            source = nodes_dict[row[f'{glct_prefix}{current_year}']]
            target = nodes_dict[row[f'{glct_prefix}{next_year}']]
            value = float(row[f'{garea_prefix}{current_year}'])
            
            # Only add link if source and target are different
            if source != target:
                link = {
                    "source": source,
                    "target": target,
                    "value": value
                }
                links.append(link)

    # Create final Sankey data structure
    sankey_data = {
        "nodes": nodes_data,
        "links": links
    }
    
    return sankey_data


# Define years
years = [1970, 2020]

# Generate Sankey data
sankey_data = generate_sankey_data(result, years)
# sankey_data

# Save to JSON file
with open('sankey_data_result_drop_overtime.json', 'w') as json_file:
    json.dump(sankey_data, json_file, indent=4)

In [ ]:
# import json

# # Daftar tahun yang dinamis
# tahun = [1970, 2020]

# # Kumpulkan semua node (nilai unik dari kolom GLCT setiap tahun)
# nodes = []
# for year_col in tahun:
#     unique_values = panelB[f"GLCT{year_col}"].unique()
#     for val in unique_values:
#         if val not in nodes:
#             nodes.append(val)

# # Membuat dictionary untuk menghubungkan node dengan indeksnya
# nodes_dict = {node: index for index, node in enumerate(nodes)}

# # Membuat links berdasarkan pasangan tahun berturut-turut
# links = []

# for i in range(len(tahun) - 1):
#     current_year = tahun[i]
#     next_year = tahun[i + 1]

#     # Loop melalui setiap baris data
#     for _, row in panelB.iterrows():
#         source = nodes_dict[row[f'GLCT{current_year}']]
#         target = nodes_dict[row[f'GLCT{next_year}']]
#         value = row[f'GAREA{current_year}']  # Anda bisa sesuaikan value dengan logika yang dibutuhkan
        
#         # Hanya tambahkan link jika source dan target berbeda
#         if source != target:
#             links.append({
#                 "source": source,
#                 "target": target,
#                 "value": value
#             })

# # Membuat data yang akan digunakan untuk Sankey diagram
# sankey_data = {
#     "nodes": [{"node": index, "name": name} for name, index in nodes_dict.items()],
#     "links": links
# }

# # Simpan ke file JSON
# with open('sankey_data_global_panelB.json', 'w') as json_file:
#     json.dump(sankey_data, json_file, indent=4)


In [48]:
import plotly.graph_objects as go

# Function to create a Sankey diagram from a dataframe
def create_sankey(df, tahun, node_scaling_factor, x, y):
    nodes = {}
    links = []
    node_colors = []
    link_colors = []

    for i in range(len(tahun) - 1):
        current_year = tahun[i]
        next_year = tahun[i + 1]

        gltct_current = f'GLCT{current_year}'
        color_current = f'color{current_year}'
        value_current = f'GAREA{current_year}'

        gltct_next = f'GLCT{next_year}'
        color_next = f'color{next_year}'
        value_next = f'GAREA{next_year}'

        for index, row in df.iterrows():
            source_node = row[gltct_current]
            target_node = row[gltct_next]
            color_source = row[color_current]

            if source_node not in nodes:
                nodes[source_node] = len(nodes)
                node_colors.append(color_source)
            if target_node not in nodes:
                nodes[target_node] = len(nodes)
                node_colors.append(row[color_next])

            link = {
                'source': nodes[source_node],
                'target': nodes[target_node],
                'value': row[value_next] * node_scaling_factor  # Scale node sizes
            }

            links.append(link)
            link_colors.append(color_source)

    return go.Sankey(
        domain=dict(
            x=x,  # Specify the x domain for positioning the plot
            y=y   # Specify the y domain for positioning the plot
        ),
        node=dict(
            pad=10,
            thickness=30,
            line=dict(color='white'),
            label=list(nodes.keys()),
            color=node_colors
        ),
        link=dict(
            source=[link['source'] for link in links],
            target=[link['target'] for link in links],
            value=[link['value'] for link in links],
            color=link_colors,
        )
    )

# Define scaling factor (assumed based on max area)
max_area = max(panelA[f'GAREA{tahun[-1]}'].max(), panelB[f'GAREA{tahun[-1]}'].max())
scaling_factor = 5 / max_area  # 5-inch max node size for the largest node

# Create a new figure
fig = go.Figure()

# Create Sankey plots for two different dataframes
sankey_plot1 = create_sankey(panelA, tahun, scaling_factor, [0.0, 0.45], [0.0, 1.0])
sankey_plot2 = create_sankey(panelB, tahun, scaling_factor, [0.55, 1.0], [0.0, 1.0])

# Add the Sankey plots to the figure
fig.add_trace(sankey_plot1)
fig.add_trace(sankey_plot2)

# Update layout with consistent annotations
fig.update_layout(
    showlegend=True,
    plot_bgcolor="rgba(0,0,0,0)",
    width=1000,
    height=600,
    annotations=[
        dict(
            text="1970", x=0, y=1.1, xref="paper", yref="paper",
            showarrow=False, arrowhead=2, ax=0, ay=-40,
            font=dict(size=16, color="black", family="Arial Black")
        ),
        dict(
            text="2020", x=1, y=1.1, xref="paper", yref="paper",
            showarrow=False, arrowhead=2, ax=0, ay=-40,
            font=dict(size=16, color="black", family="Arial Black")
        )
    ],
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    )
)

note_text = "Note: Numbers may not sum to 100% due to rounding<br>Source: Potapov et al. 2021"
fig.add_annotation(
    text=note_text,
    xref="paper", yref="paper",
    x=0,
    y=-0.2,
    showarrow=False,
    font=dict(size=10),
    align="left"
)

# Show the figure with subplots
fig.show()


In [45]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_sankey_data(panel, tahun, period_start, period_end):
    nodes = {}
    links = []
    node_colors = []
    link_colors = []

    for i in range(len(tahun) - 1):
        current_year = tahun[i]
        next_year = tahun[i + 1]

        if not (current_year == period_start and next_year == period_end):
            continue

        gltct_current = f'GLCT{current_year}'
        color_current = f'color{current_year}'
        value_current = f'GAREA{current_year}'

        gltct_next = f'GLCT{next_year}'
        color_next = f'color{next_year}'
        value_next = f'GAREA{next_year}'

        for index, row in panel.iterrows():
            source_node = row[gltct_current]
            target_node = row[gltct_next]
            color_source = row[color_current]

            if source_node not in nodes:
                nodes[source_node] = len(nodes)
                node_colors.append(color_source)
            if target_node not in nodes:
                nodes[target_node] = len(nodes)
                node_colors.append(row[color_next])

            link = {
                'source': nodes[source_node],
                'target': nodes[target_node],
                'value': row[value_next]
            }
            
            links.append(link)
            link_colors.append(color_source)
    
    return nodes, links, node_colors, link_colors

# Membuat dua figure terpisah
# Figure 1: 1970-1990
fig1 = go.Figure()

nodes1, links1, node_colors1, link_colors1 = create_sankey_data(panelA, tahun, 1970, 2020)

sankey1 = go.Sankey(
    node=dict(
        pad=20,        # Increased padding
        thickness=20,  # Adjusted thickness
        line=dict(color='white', width=0.5),
        label=list(nodes1.keys()),
        color=node_colors1,
        x=[0.01, 0.01, 0.01, 0.99],  # Positioned nodes more explicitly
        y=[0.2, 0.5, 0.8, 0.5]       # Distributed vertically
    ),
    link=dict(
        source=[link['source'] for link in links1],
        target=[link['target'] for link in links1],
        value=[link['value'] for link in links1],
        color=link_colors1
    )
)

fig1.add_trace(sankey1)

# Figure 2: 2000-2020
fig2 = go.Figure()

nodes2, links2, node_colors2, link_colors2 = create_sankey_data(panelB, tahun, 1970, 2020)

sankey2 = go.Sankey(
    node=dict(
        pad=20,        # Increased padding
        thickness=20,  # Adjusted thickness
        line=dict(color='white', width=0.5),
        label=list(nodes2.keys()),
        color=node_colors2,
        x=[0.01, 0.99, 0.99, 0.99, 0.99, 0.99],  # Positioned nodes more explicitly
        y=[0.5, 0.1, 0.3, 0.5, 0.7, 0.9]         # Distributed vertically
    ),
    link=dict(
        source=[link['source'] for link in links2],
        target=[link['target'] for link in links2],
        value=[link['value'] for link in links2],
        color=link_colors2
    )
)

fig2.add_trace(sankey2)

# Membuat legend items
label = ['Agriculture', 'Forested Natural Vegetation', "Grassland", 'Non-Forested Natural Vegetation']
warna = ['#FF6600','#347928','#FCCD2A','#B7E0FF']

# Update layout untuk fig1
fig1.update_layout(
    title=dict(
        text="Land Cover Change To Cropland (2003-2019)",
        x=0.5,
        y=0.98,
        xanchor='center',
        yanchor='top',
        font=dict(size=16)
    ),
    font_size=12,
    plot_bgcolor="rgba(0,0,0,0)",
    width=600,     # Adjusted width
    height=400,    # Adjusted height
    margin=dict(
        t=80,      # Top margin
        l=100,     # Left margin
        r=100,     # Right margin
        b=80       # Bottom margin
    ),
    paper_bgcolor='white'
)

# Update layout untuk fig2
fig2.update_layout(
    title=dict(
        text="Land Cover Change From Cropland (2003-2019)",
        x=0.5,
        y=0.98,
        xanchor='center',
        yanchor='top',
        font=dict(size=16)
    ),
    font_size=12,
    plot_bgcolor="rgba(0,0,0,0)",
    width=600,     # Adjusted width
    height=400,    # Adjusted height
    margin=dict(
        t=80,      # Top margin
        l=100,     # Left margin
        r=100,     # Right margin
        b=80       # Bottom margin
    ),
    paper_bgcolor='white'
)

note_text = "Note: Numbers may not sum to 100% due to rounding<br>Source: Potapov et al. 2021"
fig1.add_annotation(
    text=note_text,
    xref="paper", yref="paper",
    x=0,
    y=-0.2,
    showarrow=False,
    font=dict(size=10),
    align="left"
)

fig2.add_annotation(
    text=note_text,
    xref="paper", yref="paper",
    x=0,
    y=-0.2,
    showarrow=False,
    font=dict(size=10),
    align="left"
)

# Menampilkan kedua figure
fig1.show()
fig2.show()

In [15]:
nodes = {}
links = []
node_colors = []
link_colors = []

for i in range(len(tahun) - 1):
    current_year = tahun[i]
    next_year = tahun[i + 1]

    gltct_current = f'GLCT{current_year}'
    color_current = f'color{current_year}'
    value_current = f'GAREA{current_year}'

    gltct_next = f'GLCT{next_year}'
    color_next = f'color{next_year}'
    value_next = f'GAREA{next_year}'

    for index, row in df_filtered1.iterrows():
        source_node = row[gltct_current]
        target_node = row[gltct_next]
        color_source = row[color_current]

        if source_node not in nodes:
            nodes[source_node] = len(nodes)
            node_colors.append(color_source)
        if target_node not in nodes:
            nodes[target_node] = len(nodes)
            node_colors.append(row[color_next])

        link = {
            'source': nodes[source_node],
            'target': nodes[target_node],
            'value': row[value_next]
        }
        
        links.append(link)
        link_colors.append(color_source)

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=10,
        thickness=30,
        line=dict(color='white'),
        #label=list(nodes.keys()),
        color=node_colors
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=link_colors,
    
    )
)])

custom_labels = ['Agriculture', 'Forested Natural Vegetation', 'Non-Forested Natural Vegetation', "Grassland"]

nodes_1970 = [key for key in nodes.keys() if "1970" in key]
nodes_2020 = [key for key in nodes.keys() if "2020" in key]

label = ['Agriculture', 'Forested Natural Vegetation', "Grassland", 'Non-Forested Natural Vegetation']
warna = ['#FF6600','#347928','#FCCD2A','#B7E0FF']

legend_entries = []
for color, node in zip(warna, label):
    legend_entries.append([color, node])

legend = []
for i, entry in enumerate(legend_entries):
    legend.append(
        go.Scatter(
            mode="markers",
            x=[None],
            y=[None],
            marker=dict(size=10, color=entry[0], symbol="square"),
            name=entry[1],
            showlegend=True
        )
    )

# Create traces
traces = [fig.data[0]] + legend

# Define the layout with annotations included
layout = go.Layout(
    showlegend=True,
    plot_bgcolor="rgba(0,0,0,0)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    ),
    annotations=[
        # Simplified custom text
        dict(
            text="1970",
            x=0,  # Centered for visibility
            y=1.1,    # Adjust y to ensure it's within bounds
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=16, color="black",family="Arial Black")
        ),
        dict(
            text="2020",
            x=1,
            y=1.1,  # Adjusted for clarity
            xref="paper",
            yref="paper",
            showarrow=False,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=16, color="black",family="Arial Black")
        )
    ]
)

# Update the figure with traces and layout
fig = go.Figure(data=traces, layout=layout)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    width=1000,
    height=600
)

# Finally show the figure
fig.show()
